# Copernicus Berlin UV & Pollen Risk Score

This notebook fetches UV index and pollen forecast data for Berlin, computes a user-weighted risk score, and visualizes the result.

**Instructions:**
- Run each cell in order.
- Adjust the weights in the cell below to match your user profile.
- The notebook is ready for use in the Copernicus ADS Toolbox JupyterLab.

---

**Note:** The forecast window is set to 5 days (today + 4 days) to match the maximum available from Copernicus.

In [ ]:
!pip install cdsapi xarray numpy matplotlib

In [ ]:
# User-adjustable weights
UV_WEIGHT = 4  # strong sensitivity
POLLEN_WEIGHT = 2  # slight allergy

# Normalization constants
UV_MAX = 11
POLLEN_MAX = 100

# Berlin bounding box: [North, West, South, East]
BERLIN_AREA = [53.5, 12.0, 51.5, 14.5]

In [ ]:
import cdsapi
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import os

# Set up date range for 5 days (today + 4 days)
start_date = datetime.now().strftime('%Y-%m-%d')
end_date = (datetime.now() + timedelta(days=4)).strftime('%Y-%m-%d')

# Create a data directory if not present
os.makedirs('data', exist_ok=True)

In [ ]:
# Fetch UV Index (CAMS global atmospheric composition forecasts)
c = cdsapi.Client()
uv_path = 'data/uv_berlin.nc'
if not os.path.exists(uv_path):
    c.retrieve(
        'cams-global-atmospheric-composition-forecasts',
        {
            'date': f'{start_date}/{end_date}',
            'type': 'forecast',
            'format': 'netcdf',
            'variable': 'uv_index',
            'model_level': '60',
            'time': [
                '00:00', '03:00', '06:00', '09:00', '12:00', '15:00', '18:00', '21:00',
            ],
            'area': BERLIN_AREA,
        },
        uv_path)
print('UV index data ready.')

In [ ]:
# Fetch Pollen (CAMS European air quality forecasts)
pollen_path = 'data/pollen_berlin.nc'
pollen_vars = [
    'birch_pollen',
    'grass_pollen',
    'olive_pollen',
    'ragweed_pollen',
]
if not os.path.exists(pollen_path):
    c.retrieve(
        'cams-europe-air-quality-forecasts',
        {
            'date': f'{start_date}/{end_date}',
            'type': 'forecast',
            'format': 'netcdf',
            'variable': pollen_vars,
            'model_level': '0',
            'time': [
                '00:00', '03:00', '06:00', '09:00', '12:00', '15:00', '18:00', '21:00',
            ],
            'area': BERLIN_AREA,
        },
        pollen_path)
print('Pollen data ready.')

In [ ]:
# Load UV data
uv_ds = xr.open_dataset(uv_path)
uv = uv_ds['uv_index']
print(uv)
# Load pollen data (sum all available types for a total pollen score)
pollen_ds = xr.open_dataset(pollen_path)
pollen = sum([pollen_ds[var] for var in pollen_vars if var in pollen_ds])
print(pollen)

In [ ]:
# Align time and grid
# We'll use the first time step as an example
uv_t0 = uv.isel(time=0)
pollen_t0 = pollen.isel(time=0)

# Normalize
uv_norm = uv_t0 / UV_MAX
pollen_norm = pollen_t0 / POLLEN_MAX

# Risk score
risk_score = (uv_norm * UV_WEIGHT + pollen_norm * POLLEN_WEIGHT) / (UV_WEIGHT + POLLEN_WEIGHT) * 10
risk_score = risk_score.clip(1, 10)

# Plot
plt.figure(figsize=(8, 6))
risk_score.plot(cmap='RdYlGn_r', vmin=1, vmax=10, cbar_kwargs={'label': 'Risk Score (1-10)'})
plt.title('Berlin Risk Score (UV + Pollen)')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.show()

---
**You can extend this notebook to loop over all time steps, export data, or create interactive maps using `hvplot` or `folium` if desired.**